## **Data Preprocessing**

In [ ]:
import pandas as pd
from datetime import datetime
from langdetect import detect
import numpy as np
import os
import re

from sentence_transformers import SentenceTransformer

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances

In [ ]:
STATUS_LOG_PATH     = '../../data/status_log.csv"
TICKETS_PATH        = '../../data/tickets.csv"
TICKETS_FOLDER_PATH = '../../data/tickets/"
FAQ_QUESTIONS_PATH  = '../../data/FAQ_questions.txt"

FINAL_TICKET_NAME   = '../../data/tickets_postprp.pkl'

QUESTIONS_TRANSFORMER = "bert-base-nli-mean-tokens"

MAX_LEN = 256

BERT_MODEL = 'bert-base-multilingual-cased'

In [ ]:
df = pd.read_csv(TICKETS_PATH)
df.head()

,ID,Beschreibung,Kategorie ID,Kategorietext,Unterkategorie ID,Unterkategorietext,Status,Angelegt Am,Angelegt Von,Geändert Am,Geändert Von,Auftraggeber,Meldender,Support Team,Bearbeiter,Nr. Nachrichten,Erste Antwort,Letzte Antwort
0,2000000060,,ZSD_SR_AA,Allgemeine Anfragen,ZSD_SR_AA_SON,Sonstiges,E0008 Quittiert,2015.11.25\t18:35:21,H2451TK1,2015.12.18\t02:00:47,H2851RM1,H2451,H2451TK1,0000000046,H2851RM1,5,2015.11.25\t19:35:19,2015.11.26\t14:26:08
1,2000000070,,ZSD_SR_BO,SAP Netweaver Business Objects,SR_ZSD_BO_FP,Fallstudienproblem,E0008 Quittiert,2015.11.26\t13:49:29,H2741CR1,2016.03.08\t09:59:56,H2851AS9,H2741,H2741CR1,0000000046,NaN,5,2015.11.26\t14:49:16,2015.11.26\t16:08:45
2,2000000071,,ZSD_SR_BO,SAP Netweaver Business Objects,SR_ZSD_BO_FP,Fallstudienproblem,E0008 Quittiert,2015.11.26\t14:37:47,H2741CR1,2016.05.31\t16:00:40,BORYS,H2741,H2741CR1,0000000047,H2851LP2,5,2015.11.26\t15:37:34,2016.05.31\t18:00:40
3,2000000073,,ZSD_SR_BYD,SAP Business by Design,ZSD_SR_BYD_PRM,Passwortrücksetzung für Masteruser,E0008 Quittiert,2015.11.26\t17:40:10,H2851GB2,2016.05.31\t16:00:40,BORYS,H2851,H2851GB2,0000000047,H2851GB2,1,2016.05.31\t18:00:40,2016.05.31\t18:00:40
4,2000000080,,ZSD_SR_GBI,SAP ERP GBI,ZSD_SR_GBI_SON,Sonstiges,E0008 Quittiert,2015.11.30\t10:15:00,H2702AR1,2015.12.18\t14:40:35,H2702AR1,H2702,H2702AR1,0000000044,H2851RH3,5,2015.11.30\t11:14:29,2015.12.16\t17:14:56


**Define Functions for Data Preprocessing**

In [ ]:
# Categories for Support Levels
first_level_subcat  = ["Mandantenrücksetzung ", "Entwicklerschlüssel beantragen ", "Freischaltung einer IP Adresse/Range ", \
                       "Passwortrücksetzung für Masteruser ", "Remote Login ", "H/U-User für Dozenten ", \
                       "EMEA Portal / UAC ", "ERPsim BI ", "Stammdaten/Ansprechpartner ", \
                       "S-User ", "H/U-User für Studenten ", \
                       "Passwortrücksetzung für ERPSim-User ", "ERPsim Demo-System ", "ERPsim Kursmanagement ", \
                       "ERPsim Qualifizierung "]

first_level_cat     = ["SAP GUI"]     

second_level_subcat = ["System Erreichbarkeit ", "Systemverfügbarkeit ", "Systemrücksetzung ", "Systemfehler ", \
                       "Systemkonfiguration /Berechtigungsprobleme ", "Fallstudienproblem ", "TERP10 Anfrage ", \
                       "GBI Fallstudienproblem ", "GBI/GBS Fallstudienproblem ", "Testzugang/Pilot ", "Vertrag ", \
                       "TERP10/TS410 Anfrage ", "Rechnung ", "ERPsim Zahlungsmodalitäten ", "Schulungen "]

second_level_cat    = ["SAP NW", "SAP Netweaver", "TERP 10", "TERP10/TS410", "SAP for Healthcare", "SAP4school", \
                       "SAP Business Warehouse", "sap4school", "erp4school"]

In [ ]:
def get_editors(ticket_df):
  i = 0
  editors = ""
  while i < ticket_df.shape[0]:
    #if ticket_df.loc[i, "Absender"].strip() != df.loc[row, "Meldender"] and df.loc[row, "editors"] is None:
    if ticket_df.loc[i, "Absender"].strip() != df.loc[row, "Meldender"] and editors == "":
      editors += str(ticket_df.loc[i, "Absender"]) + ";"
      i += 1
    #elif ticket_df.loc[i, "Absender"].strip() != df.loc[row, "Meldender"] and not ticket_df.loc[i, "Absender"].strip() in df.loc[row, "editors"]:
    elif ticket_df.loc[i, "Absender"].strip() != df.loc[row, "Meldender"] and not ticket_df.loc[i, "Absender"].strip() in editors:   
      editors += str(ticket_df.loc[i, "Absender"]) + ";"
      i += 1
    else:
      i += 1
  
  return editors

def get_num_editors(df):
  df["num_editors"] = df['editors'].str.count(';')
  df["num_editors"].fillna(0.0, inplace=True)
  df["num_editors"] = df["num_editors"].astype(int)

  return df

def get_time_parameters(df):
  df_status_logs = pd.read_csv(STATUS_LOG_PATH)

  df_quit = df_status_logs[df_status_logs["Status Text"] == "Quittiert"].reset_index()
  df_neu = df_status_logs[df_status_logs["Status Text"] == "Neu"].reset_index()

  df_quit = df_quit[df_quit["ID"].isin(df_neu["ID"].to_list())]

  df_neu = df_neu[df_neu["ID"].isin(df_quit["ID"].to_list())]

  df_neu["time"] = df_neu["Datum"] + " " + df_neu["Uhrzeit"]
  df_quit["time"] = df_quit["Datum"] + " " + df_quit["Uhrzeit"]

  df_neu["time_start"] = df_neu["time"].apply(lambda s: datetime.strptime(s, '%Y.%m.%d %H:%M:%S'))
  df_quit["time_finish"] = df_quit["time"].apply(lambda s: datetime.strptime(s, '%Y.%m.%d %H:%M:%S'))

  df_quit.drop(["index", "Datum", "Uhrzeit", "Geändert Von", "Status ID", "Status Text", "time"], axis=1, inplace=True)
  df_neu.drop(["index", "Datum", "Uhrzeit", "Geändert Von", "Status ID", "Status Text", "time"], axis=1, inplace=True)

  df_status_logs = pd.merge(df_neu, df_quit, on='ID')

  df_status_logs["time_taken"] = df_status_logs["time_finish"] - df_status_logs["time_start"]

  df_status_logs["time_taken"] = df_status_logs["time_taken"].apply(lambda t: (t.total_seconds())/3600)

  df = pd.merge(df, df_status_logs, how='outer', on=['ID'])
  
  return df

def get_initial_messages(ticket_df):
  first_sender = ticket_df.Absender[0]

  initial_message_bool = True

  initial_message = ""
  #description     = ""
  #answer          = ""
  #internal_note   = ""   
  
  for i, message in ticket_df.iterrows():
    if message.Text == None or message.Text == "":
      continue

    temp = str(message.Text).replace("\n"," ") #replace line breaks with whitespace
    temp = re.sub("<[a-zA-Z0-9_]*>", "Mr. Smith", temp) #get rid of anonymity brackets

    if not message.Absender == first_sender:
      initial_message_bool = False

    # systemdaten is not relevant
    if initial_message_bool and not message.Nachrichtentyp == "Systemdaten ":
      initial_message += " " + temp

    #if message.Nachrichtentyp == "Beschreibung ":
    #  description += " " + temp

    #if message.Nachrichtentyp == "Antwort ":
    #  answer += " " + temp

    #if message.Nachrichtentyp == "Interne Notiz ":
    #  internal_note += " " + temp      
 
  return initial_message

def get_language_flag(df):
  # remove initial messages with no text or just numbers
  # otherwise no language can be detected
  initial_messages = df.initial_message
  df = df[df['initial_message'].notna()]

  language_flag = []
  for i, message in enumerate(df.initial_message):
    try:
      language_flag.append(detect(message))
    except:
      language_flag.append('error')
  
  df["language"] = language_flag
  df = df[df["language"] != "error"]

  return df
  
def get_question_embeddings(transformer):
  questions = []
  f = open(FAQ_QUESTIONS_PATH, "r")
  questions_raw = []
  for line in f:
    questions_raw.append(line)    
  f.close()

  for line in questions_raw:
    temp = line.replace("\ufeff","")
    temp = temp.replace("\n","")
    questions.append(temp)

  questions_embeddings = transformer.encode(questions)

  return questions_embeddings

def get_max_similarity(similarity):
  max_similarity = np.min(similarity)
  faq_index = np.argmin(similarity).astype(int)

  return max_similarity, faq_index

def get_min_difference(difference):
  min_difference = np.min(difference)
  faq_index = np.argmin(difference).astype(int)

  return min_difference, faq_index

def get_support_lvl_editors(df):
  df['support_lvl_editors'] = 0
  df.loc[df['num_editors'] <= 1, ['support_lvl_editors']] = 1
  df.loc[df['num_editors'] > 1, ['support_lvl_editors']] = 2

  return df

def get_support_lvl_cat(df):
  labels = []

  for index, row in df.iterrows():
    #first level task
    if any(subcat in str(row["Unterkategorietext"]) for subcat in first_level_subcat) or any(cat in str(row["Kategorietext"]) for cat in first_level_cat):
      labels.append(1)
    else:
    
    #second level task
      if any(subcat in str(row["Unterkategorietext"]) for subcat in second_level_subcat) or any(cat in str(row["Kategorietext"]) for cat in second_level_cat):
        labels.append(2)
      #otherwise: don't use row
      else:
        labels.append(99)

  df["support_lvl_cat"] = labels
  df["support_lvl_cat"] = df["support_lvl_cat"].astype(int)
  
  return df

**Transform Data**

In [ ]:
# Prepare Transformer and FAQ Question Embeddings
transformer = SentenceTransformer(QUESTIONS_TRANSFORMER)
questions_embeddings = get_question_embeddings(transformer=transformer)

tokenizer = BertTokenizer.from_pretrained(BERT_MODEL)

df = pd.read_csv(TICKETS_PATH)

# Create column 'editors', 'initial_message' that contains all users that worked on the ticket
df["editors"]           = None
df["initial_message"]   = None
df["similarity"]        = None
df["difference"]        = None
df["embedding"]         = None
df["max_similarity"]    = None
df["min_difference"]    = None
df["faq_index_max_sim"] = None
df["faq_index_min_dif"] = None
df["tokens"]            = None

# Impute columns with missing values
df["Kategorie ID"].replace({" ": "AUTOMATED"}, inplace=True)
df["Unterkategorie ID"].replace({" ": "AUTOMATED"}, inplace=True)
df["Kategorietext"].replace({" ": "AUTOMATED"}, inplace=True)
df["Unterkategorietext"].replace({" ": "AUTOMATED"}, inplace=True)
df["Meldender"].replace({" ": "EMPTY"}, inplace=True)
df["Auftraggeber"].replace({" ": "EMPTY"}, inplace=True)
df["Angelegt Am"] = pd.to_datetime(df["Angelegt Am"])
df["Angelegt Am"] = df["Angelegt Am"].apply(lambda x: x.timestamp())

for row in range(df.shape[0]):
  if row % 100 is 0:
    print(row)
  path = TICKETS_FOLDER_PATH + str(df.loc[row, "ID"]) + ".csv"
  try:
    ticket_df = pd.read_csv(path)
  except:
    continue

  # Add editors to tickets
  editors = get_editors(ticket_df)
  initial_message = get_initial_messages(ticket_df)

  if initial_message is not None:
    embedding  = transformer.encode(str(initial_message))
    similarity = cosine_similarity([embedding], questions_embeddings[:])
    difference = euclidean_distances([embedding], questions_embeddings[:])

    max_similarity, faq_index_max_sim = get_max_similarity(similarity)
    min_difference, faq_index_min_dif = get_min_difference(difference)

    df.at[row,"editors"]   = editors
    df.at[row,"initial_message"]   = initial_message
    df.at[row,'similarity']        = similarity
    df.at[row,'difference']        = difference
    df.at[row,'embedding']         = embedding
    df.at[row,'max_similarity']    = max_similarity
    df.at[row,'min_difference']    = min_difference
    df.at[row,'faq_index_max_sim'] = faq_index_max_sim
    df.at[row,'faq_index_min_dif'] = faq_index_min_dif 

#df.to_csv(f"{FINAL_TICKET_NAME}intermediate.csv")

#df = pd.load_csv(f"{FINAL_TICKET_NAME}intermediate.csv")

# Calc num editors 
df = get_num_editors(df)

# Calc time parameters
df = get_time_parameters(df)

# Add language flag
df = get_language_flag(df)

#Add support level based on number of editors
df = get_support_lvl_editors(df)

# Add support level based on categories
df = get_support_lvl_cat(df)

df.drop(columns = ['Geändert Am', 'Geändert Von', 'Support Team', 'Erste Antwort', 'Letzte Antwort'], inplace = True)

df.rename(columns={'ID': 'id', 
                   'Beschreibung': 'beschreibung', 
                   'Kategorie ID': 'kategorie_id', 
                   'Kategorietext': 'kategorietext', 
                   'Unterkategorie ID': 'unterkategorie_id', 
                   'Unterkategorietext': 'unterkategorietext', 
                   'Status': 'status', 
                   'Angelegt Am': 'angelegt_am', 
                   'Angelegt Von': 'angelegt_von', 
                   'Auftraggeber': 'auftraggeber', 
                   'Meldender': 'meldender', 
                   'Bearbeiter': 'bearbeiter', 
                   'Nr. Nachrichten': 'num_messages'}, inplace=True)

df['max_similarity'] = df['max_similarity'].astype(float)
df['min_difference'] = df['min_difference'].astype(float)
df['faq_index_max_sim'] = df['faq_index_max_sim'].astype(int)
df['faq_index_min_dif'] = df['faq_index_min_dif'].astype(int)

df.to_csv(f"{FINAL_TICKET_NAME}.csv")

df.to_pickle(f"{FINAL_TICKET_NAME}.pkl")


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


**Check Final Output**

In [ ]:
df.head()

,id,beschreibung,kategorie_id,kategorietext,unterkategorie_id,unterkategorietext,status,angelegt_am,angelegt_von,auftraggeber,meldender,bearbeiter,num_messages,editors,initial_message,similarity,difference,embedding,max_similarity,min_difference,faq_index_max_sim,faq_index_min_dif,num_editors,time_start,time_finish,time_taken,language,support_lvl_editors,support_lvl_cat
0,2000000060,,ZSD_SR_AA,Allgemeine Anfragen,ZSD_SR_AA_SON,Sonstiges,E0008 Quittiert,1.448477e+09,H2451TK1,H2451,H2451TK1,H2851RM1,5,H2851DP2;H2851RM4;,"Hallo zusammen aktuell sind die Systeme i06,...","[[0.7254113, 0.64478034, 0.75147295, 0.7483704...","[[11.533532, 13.114959, 11.112994, 10.836821, ...","[0.14102867, 0.87266093, 1.3615568, 0.51052284...",0.591029,9.864998,12,14,2,2015-11-25 22:09:11,2015-12-18 03:00:47,532.860000,de,2,99
1,2000000070,,ZSD_SR_BO,SAP Netweaver Business Objects,SR_ZSD_BO_FP,Fallstudienproblem,E0008 Quittiert,1.448546e+09,H2741CR1,H2741,H2741CR1,NaN,5,,"Ich weiß nicht, woher ich die SW Information ...","[[0.75585, 0.6791788, 0.7309612, 0.7704123, 0....","[[10.914412, 12.509426, 11.593696, 10.391296, ...","[-0.34037295, 1.1536975, 0.8353497, 0.52754015...",0.645721,9.467989,12,18,0,2015-11-26 16:03:04,2016-03-08 10:59:56,2466.947778,de,1,2
2,2000000071,,ZSD_SR_BO,SAP Netweaver Business Objects,SR_ZSD_BO_FP,Fallstudienproblem,E0008 Quittiert,1.448549e+09,H2741CR1,H2741,H2741CR1,H2851LP2,5,BORYS;,Da ich keine neue Connection anlegen kann (is...,"[[0.7678155, 0.6977639, 0.78477746, 0.8256672,...","[[10.632739, 12.12854, 10.368579, 9.043479, 10...","[-0.33647916, 1.0358676, 1.183024, 0.33042786,...",0.666763,7.828650,12,14,1,2015-11-26 17:59:58,2016-05-31 18:00:40,4488.011667,de,1,2
3,2000000073,,ZSD_SR_BYD,SAP Business by Design,ZSD_SR_BYD_PRM,Passwortrücksetzung für Masteruser,E0008 Quittiert,1.448560e+09,H2851GB2,H2851,H2851GB2,H2851GB2,1,BORYS;,"Meldung automatisch durch Report ""AI_SDK_SP_A...","[[0.66387796, 0.5765782, 0.6556866, 0.6701319,...","[[12.662242, 14.205878, 12.973542, 12.3057785,...","[-0.1209006, 0.41539708, 1.6555126, 0.09115197...",0.552797,11.220666,12,17,1,2015-11-27 09:31:03,2016-05-31 18:00:40,4472.493611,de,1,1
4,2000000080,,ZSD_SR_GBI,SAP ERP GBI,ZSD_SR_GBI_SON,Sonstiges,E0008 Quittiert,1.448878e+09,H2702AR1,H2702,H2702AR1,H2851RH3,5,H2851RH3;,"Guten Morgen, SAP UCC Team, ich würde gerne ...","[[0.6271755, 0.6238837, 0.580957, 0.6047468, 0...","[[13.260491, 13.322233, 14.224812, 13.388447, ...","[-0.21527165, 1.3987509, 0.267509, 0.24152803,...",0.526619,9.676493,9,17,1,2015-12-03 14:12:58,2015-12-18 15:40:35,361.460278,de,1,99
